In [16]:
import pandas as pd
import numpy as np
from scipy.fft import fft
from typing import Dict
from datetime import datetime

class CornealMetricsAnalyzer:
    def __init__(self):
        self.setup_thresholds()
    
    def setup_thresholds(self):
        """Setup research-based clinical thresholds."""
        self.clinical_ranges = {
            'central_power': {
                'normal': (43.94 - 9.78, 43.94 + 9.78),  # Research-based
                'suspect': (43.94 - 12, 43.94 + 12)
            },
            'regularity': {
                'normal': 0.7,
                'suspect': 0.5
            },
            'symmetry': {
                'normal': 0.88,
                'suspect': 0.75
            }
        }

    def analyze_cornea(self, df: pd.DataFrame) -> Dict:
        """Comprehensive corneal analysis returning only metrics."""
        # Basic spectral analysis
        spectral_metrics = self.calculate_spectral_metrics(df)
        
        # Regional analysis
        regional_metrics = self.analyze_regions(df)
        
        # Symmetry analysis
        symmetry_metrics = self.calculate_symmetry_metrics(df)
        
        # Clinical classification
        clinical_status = self.determine_clinical_status(
            spectral_metrics,
            regional_metrics,
            symmetry_metrics
        )
        
        return {
            'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'spectral_metrics': spectral_metrics,
            'regional_metrics': regional_metrics,
            'symmetry_metrics': symmetry_metrics,
            'clinical_status': clinical_status
        }

    def calculate_spectral_metrics(self, df: pd.DataFrame) -> Dict:
        """Calculate spectral analysis metrics."""
        k_matrix = df.pivot_table(
            values='Keratometry_Value',
            index='Meridian_Index',
            columns='Radial_Index'
        ).values

        # Meridional analysis
        meridian_fft = fft(k_matrix, axis=0)
        meridian_power = np.abs(meridian_fft)**2
        meridian_power = np.mean(meridian_power, axis=1)
        meridian_power = meridian_power / np.sum(meridian_power)

        # Radial analysis
        radial_fft = fft(k_matrix, axis=1)
        radial_power = np.abs(radial_fft)**2
        radial_power = np.mean(radial_power, axis=0)
        radial_power = radial_power / np.sum(radial_power)

        return {
            'meridional_dc': meridian_power[0],
            'radial_dc': radial_power[0],
            'meridional_irregularity': 1 - meridian_power[0],
            'radial_irregularity': 1 - radial_power[0],
            'high_order_ratio': np.sum(meridian_power[3:]) / np.sum(meridian_power)
        }

    def analyze_regions(self, df: pd.DataFrame) -> Dict:
        """Regional analysis metrics."""
        regions = {
            'central': df[df['Normalized_Radius'] <= 0.3],
            'mid_peripheral': df[(df['Normalized_Radius'] > 0.3) & 
                               (df['Normalized_Radius'] <= 0.7)],
            'peripheral': df[df['Normalized_Radius'] > 0.7]
        }
        
        metrics = {}
        for name, region_df in regions.items():
            values = region_df['Keratometry_Value']
            metrics[name] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'range': np.ptp(values),
                'regularity': 1 - (np.std(values) / np.mean(values))
            }
        
        return metrics

    def calculate_symmetry_metrics(self, df: pd.DataFrame) -> Dict:
        """Calculate symmetry-based metrics."""
        angles = np.sort(df['Meridian_Angle_Deg'].unique())
        opposite_pairs = []
        
        for angle in angles[:len(angles)//2]:
            opposite_angle = (angle + 180) % 360
            angle_data = df[df['Meridian_Angle_Deg'] == angle]['Keratometry_Value']
            opposite_data = df[df['Meridian_Angle_Deg'] == opposite_angle]['Keratometry_Value']
            
            if not angle_data.empty and not opposite_data.empty:
                similarity = 1 - abs(np.mean(angle_data) - np.mean(opposite_data)) / np.mean(angle_data)
                opposite_pairs.append(similarity)

        return {
            'meridional_symmetry': np.mean(opposite_pairs),
            'radial_uniformity': self.calculate_radial_uniformity(df),
            'pattern_regularity': np.min(opposite_pairs) if opposite_pairs else 0
        }

    def calculate_radial_uniformity(self, df: pd.DataFrame) -> float:
        """Calculate uniformity of radial measurements."""
        radial_groups = df.groupby('Radial_Index')['Keratometry_Value'].agg(['mean', 'std'])
        return 1 - np.mean(radial_groups['std'] / radial_groups['mean'])

    def determine_clinical_status(self, spectral: Dict, 
                                regional: Dict, 
                                symmetry: Dict) -> Dict:
        """Determine clinical status based on metrics."""
        central_power = regional['central']['mean']
        
        status = {
            'power_status': self._classify_value(
                central_power, 
                self.clinical_ranges['central_power']
            ),
            'regularity_status': 'normal' if regional['central']['regularity'] > 
                                self.clinical_ranges['regularity']['normal'] else 'suspect',
            'symmetry_status': 'normal' if symmetry['meridional_symmetry'] > 
                             self.clinical_ranges['symmetry']['normal'] else 'suspect'
        }
        
        status['overall_status'] = ('normal' if all(s == 'normal' for s in status.values()) 
                                  else 'irregular' if any(s == 'irregular' for s in status.values())
                                  else 'suspect')
        
        return status

    def _classify_value(self, value: float, ranges: Dict) -> str:
        """Classify a value based on provided ranges."""
        if ranges['normal'][0] <= value <= ranges['normal'][1]:
            return 'normal'
        elif ranges['suspect'][0] <= value <= ranges['suspect'][1]:
            return 'suspect'
        return 'irregular'

    def save_metrics(self, analysis_results: Dict, filename: str = "corneal_metrics"):
        """Save analysis results to file."""
        report = [
            "CORNEAL METRICS ANALYSIS REPORT",
            f"Date: {analysis_results['timestamp']}\n",
            "SPECTRAL METRICS:",
            f"Meridional DC: {analysis_results['spectral_metrics']['meridional_dc']:.4f}",
            f"Radial DC: {analysis_results['spectral_metrics']['radial_dc']:.4f}",
            f"High Order Ratio: {analysis_results['spectral_metrics']['high_order_ratio']:.4f}\n",
            "REGIONAL METRICS:",
            f"Central Mean: {analysis_results['regional_metrics']['central']['mean']:.2f}D",
            f"Central Regularity: {analysis_results['regional_metrics']['central']['regularity']:.4f}\n",
            "SYMMETRY METRICS:",
            f"Meridional Symmetry: {analysis_results['symmetry_metrics']['meridional_symmetry']:.4f}",
            f"Pattern Regularity: {analysis_results['symmetry_metrics']['pattern_regularity']:.4f}\n",
            "CLINICAL STATUS:",
            f"Overall Status: {analysis_results['clinical_status']['overall_status']}"
        ]
        
        with open(f"{filename}.txt", "w") as f:
            f.write("\n".join(report))
        
        # Also save raw data for further analysis
        np.save(f"{filename}_raw.npy", analysis_results)

def main():
    analyzer = CornealMetricsAnalyzer()
    
    # File path
    file_path = '/home/aricept094/mydata/testrm_converted.csv'
    
    # Analyze data
    df = pd.read_csv(file_path)
    results = analyzer.analyze_cornea(df)
    
    # Save results
    analyzer.save_metrics(results)
    
    # Print key metrics
    print("\nKey Metrics Summary:")
    print(f"Central Power: {results['regional_metrics']['central']['mean']:.2f}D")
    print(f"Regularity Index: {results['regional_metrics']['central']['regularity']:.4f}")
    print(f"Symmetry Score: {results['symmetry_metrics']['meridional_symmetry']:.4f}")
    print(f"Clinical Status: {results['clinical_status']['overall_status']}")

if __name__ == "__main__":
    main()


Key Metrics Summary:
Central Power: 44.09D
Regularity Index: 0.9714
Symmetry Score: 0.9917
Clinical Status: normal
